In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm

In [ ]:
os.getcwd()

In [ ]:
df_train = '/kaggle/input/realwaste-dataset/splited dataset/train'
df_val = '/kaggle/input/realwaste-dataset/splited dataset/val'
df_test = '/kaggle/input/realwaste-dataset/splited dataset/test'

In [37]:
from sklearn.utils.class_weight import compute_class_weight

# Check class distribution to avoid imbalance issues
labels = os.listdir(df_train)
class_counts = {label: len(os.listdir(os.path.join(df_train, label))) for label in labels}
print("Class distribution:", class_counts)

# Compute class weights
class_weight = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(list(class_counts.keys())),
    y=np.concatenate([[label] * count for label, count in class_counts.items()])
)
class_weight = dict(enumerate(class_weight))
print("Computed class weights:", class_weight)

Class distribution: {'Metal': 474, 'Glass': 252, 'Paper': 300, 'Vegetation': 261, 'Cardboard': 276, 'Textile Trash': 190, 'Food Organics': 246, 'Plastic': 552, 'Miscellaneous Trash': 297}
Computed class weights: {0: 1.146537842190016, 1: 1.2863595302619693, 2: 1.255731922398589, 3: 0.6676043131739334, 4: 1.0654695099139544, 5: 1.0548148148148149, 6: 0.573268921095008, 7: 1.6654970760233918, 8: 1.2124308216262238}


In [38]:
import glob as gb
size = []

for label in os.listdir(df_train):
    images_paths = gb.glob(str(df_train +'/'+label+'/*.jpg'))

    for img_path in images_paths:
        img = plt.imread(img_path)
        img_shape = img.shape
        size.append(img_shape)

In [39]:
df = pd.DataFrame(size)
df.value_counts()

0    1    2
524  524  3    2848
Name: count, dtype: int64

In [40]:
size = []

for label in os.listdir(df_val):
    images_paths = gb.glob(str(df_val +'/'+label+'/*.jpg'))

    for img_path in images_paths:
        img = plt.imread(img_path)
        img_shape = img.shape
        size.append(img_shape)

In [41]:
df_va = pd.DataFrame(size)
df_va.value_counts()

0    1    2
524  524  3    949
Name: count, dtype: int64

In [42]:
size = []

for label in os.listdir(df_test):
    images_paths = gb.glob(str(df_test +'/'+label+'/*.jpg'))

    for img_path in images_paths:
        img = plt.imread(img_path)
        img_shape = img.shape
        size.append(img_shape)

In [43]:
df_tes = pd.DataFrame(size)
df_tes.value_counts()

0    1    2
524  524  3    955
Name: count, dtype: int64

train_generator

In [44]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageGenerators for training 
train_datagen = ImageDataGenerator(
    rotation_range = 15, 
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    rescale = 1/255
)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/realwaste-dataset/splited dataset/train',
    target_size=(256, 256),  #  Fix: Resizing
    batch_size = 32,
    class_mode = 'categorical'
)

Found 2848 images belonging to 9 classes.


In [45]:
train_batch = train_generator

val_generator

In [46]:
val_datagen = ImageDataGenerator(rescale = 1/255)
val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/realwaste-dataset/splited dataset/val',
    target_size=(256, 256),  #  Fix: Resizing
    batch_size = 32,
    class_mode = 'categorical'
)

Found 949 images belonging to 9 classes.


In [47]:
val_batch = val_generator

test_generator

In [48]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/realwaste-dataset/splited dataset/test',
    target_size=(256, 256),  #  Fix: Resizing
    batch_size = 32,
    class_mode = 'categorical'
)

Found 955 images belonging to 9 classes.


In [49]:
test_batch= test_generator

In [57]:
train_batch[0]

(array([[[[0.3830056 , 0.47241765, 0.60601234],
          [0.4147835 , 0.4946622 , 0.6314347 ],
          [0.41421714, 0.5007346 , 0.63529414],
          ...,
          [0.48636174, 0.53734213, 0.61185193],
          [0.48847085, 0.53945124, 0.61396104],
          [0.48475334, 0.5357337 , 0.61024356]],
 
         [[0.37995476, 0.47058827, 0.6039216 ],
          [0.40779603, 0.48977095, 0.6258447 ],
          [0.41561463, 0.5000358 , 0.63529414],
          ...,
          [0.47248212, 0.5234625 , 0.5979723 ],
          [0.47048402, 0.5214644 , 0.59253174],
          [0.4791487 , 0.5301291 , 0.59842056]],
 
         [[0.37925604, 0.47058827, 0.6039216 ],
          [0.40080854, 0.4848797 , 0.6202547 ],
          [0.41701213, 0.4993371 , 0.63529414],
          ...,
          [0.46073547, 0.5117159 , 0.5862257 ],
          [0.46545556, 0.516436  , 0.5867416 ],
          [0.4794623 , 0.5304427 , 0.5971094 ]],
 
         ...,
 
         [[0.64611274, 0.73238724, 0.87748533],
          [0.64471

In [ ]:
vgg_model = tf.keras.applications.vgg16.VGG16()

# VGG

In [50]:
from tensorflow.keras.applications import VGG19, VGG16

# 1. Load pre-trained VGG19 model (excluding top classification layer)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [51]:
# Freeze all layers first
base_model.trainable = False

In [52]:
# Unfreeze the last 5 layers
# Count total layers in base model
print(len(base_model.layers))   # for reference

# Unfreeze last 5 layers
for layer in base_model.layers[-5 : ] :
    layer.trainable = True

22


In [56]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# 2. Add custom top layers for  classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
Dropout(0.)(x)
output_layer = Dense(9, activation='softmax')(x)

In [57]:
from tensorflow.keras.layers import BatchNormalization
model.add(BatchNormalization())

AttributeError: 'Functional' object has no attribute 'add'

In [58]:
from tensorflow.keras.models import Model

# 3. add all layer togeather
model = Model(inputs=base_model.input, outputs=output_layer)

In [59]:
# import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, K.floatx())
        y_pred = K.clip(y_pred, 1e-7, 1 - 1e-7)
        loss = -y_true * alpha * K.pow(1 - y_pred, gamma) * K.log(y_pred)
        return K.sum(loss, axis=1)
    return loss

In [60]:
from tensorflow.keras.optimizers import Adam

# 4. Compile the model
model.compile(optimizer=Adam(learning_rate = 0.001), loss = focal_loss(), metrics = ['accuracy'])

In [25]:
# 5. Train the model
batch_size = 64
epochs = 10
model.fit(train_generator, epochs = epochs, validation_data = val_batch,  
          class_weight=class_weight,  # Fix: Handling class imbalance
          batch_size = batch_size)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 83s 706ms/step - accuracy: 0.1075 - loss: 0.4643 - val_accuracy: 0.1791 - val_loss: 0.3915
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 60s 621ms/step - accuracy: 0.1891 - loss: 0.3645 - val_accuracy: 0.3172 - val_loss: 0.2997
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 58s 607ms/step - accuracy: 0.3196 - loss: 0.2809 - val_accuracy: 0.3888 - val_loss: 0.2425
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 59s 616ms/step - accuracy: 0.3590 - loss: 0.2388 - val_accuracy: 0.4110 - val_loss: 0.2290
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 59s 615ms/step - accuracy: 0.4526 - loss: 0.1945 - val_accuracy: 0.4499 - val_loss: 0.1947
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 58s 606ms/step - accuracy: 0.5070 - loss: 0.1608 - val_accuracy: 0.5279 - val_loss: 0.1701
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 58s 606ms/step - accuracy: 0.5476 - loss: 0.1480 - val_accuracy: 0.5385 - val_loss: 0.1819
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 58s 609ms/step - accuracy: 0.5760 - loss: 0.1382 - val_accuracy: 0.576

In [61]:
from tensorflow.keras.optimizers import Adam

# 4. Compile the model
model.compile(optimizer=Adam(learning_rate = 0.00001), loss = focal_loss(), metrics = ['accuracy'])

In [62]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights = True)

In [63]:
# 5. Train the model
batch_size = 64
epochs = 30
model.fit(train_generator, epochs = epochs, validation_data = val_batch,
                    class_weight = class_weight,  # Fix: Handling class imbalance
                    batch_size = batch_size)
, callbacks = [early_stop]

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 66s 654ms/step - accuracy: 0.1481 - loss: 0.4319 - val_accuracy: 0.4447 - val_loss: 0.2781
Epoch 2/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 57s 599ms/step - accuracy: 0.4529 - loss: 0.2488 - val_accuracy: 0.5153 - val_loss: 0.1920
Epoch 3/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 57s 592ms/step - accuracy: 0.5697 - loss: 0.1792 - val_accuracy: 0.6143 - val_loss: 0.1569
Epoch 4/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 57s 587ms/step - accuracy: 0.6450 - loss: 0.1423 - val_accuracy: 0.6006 - val_loss: 0.1551
Epoch 5/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 57s 590ms/step - accuracy: 0.6769 - loss: 0.1268 - val_accuracy: 0.6733 - val_loss: 0.1259
Epoch 6/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 56s 588ms/step - accuracy: 0.7153 - loss: 0.1083 - val_accuracy: 0.6365 - val_loss: 0.1342
Epoch 7/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 56s 582ms/step - accuracy: 0.7226 - loss: 0.0979 - val_accuracy: 0.6754 - val_loss: 0.1192
Epoch 8/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 56s 582ms/step - accuracy: 0.7442 - loss: 0.0915 - val_accuracy: 0.676

('callbacks', '=', '[early_stop]')

In [ ]:
# 5. Train the model
batch_size = 64
epochs = 200
history = model.fit(train_generator, epochs = epochs, validation_data = val_batch, batch_size = batch_size)

overfitting // maybe use drop outs

In [109]:
y_test = test_batch.classes
y_test[:150]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [74]:
# If test_batch is a generator (like ImageDataGenerator.flow_from_directory)
y_pred_prob = model.predict(val_batch)

30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 409ms/step


In [75]:
# If using softmax, get the index of the highest predicted probability
y_pred = np.argmax(y_pred_prob, axis = 1)

# Get the true labels from the generator
y_true = val_batch.classes

In [76]:
from sklearn.metrics import classification_report

# If using flow_from_directory, use .class_indices to map labels
labels = list(val_batch.class_indices.keys())

print(classification_report(y_true, y_pred, target_names = labels))

                     precision    recall  f1-score   support

          Cardboard       0.07      0.10      0.08        92
      Food Organics       0.08      0.07      0.08        82
              Glass       0.11      0.10      0.10        84
              Metal       0.23      0.30      0.26       158
Miscellaneous Trash       0.10      0.06      0.07        99
              Paper       0.09      0.11      0.10       100
            Plastic       0.21      0.14      0.17       184
      Textile Trash       0.06      0.06      0.06        63
         Vegetation       0.06      0.06      0.06        87

           accuracy                           0.13       949
          macro avg       0.11      0.11      0.11       949
       weighted avg       0.13      0.13      0.13       949



there is something wrong here **

In [67]:
from tensorflow.keras.applications import EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [130]:
# Freeze all layers first
base_model.trainable = False

In [68]:
# Unfreeze the last 5 layers
# Count total layers in base model
print(len(base_model.layers))   # for reference

# Unfreeze last 5 layers
for layer in base_model.layers[-5 : ] :
    layer.trainable = True

385


In [69]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# 2. Add custom top layers for  classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
Dropout(0.5)(x)
output_layer = Dense(9, activation='softmax')(x)

In [70]:
from tensorflow.keras.models import Model

# 3. add all layer togeather
model = Model(inputs=base_model.input, outputs=output_layer)

In [71]:
from tensorflow.keras.optimizers import Adam

# 4. Compile the model
model.compile(optimizer=Adam(learning_rate = 0.001),  loss = focal_loss(), metrics = ['accuracy'])

In [72]:
from tensorflow.keras.callbacks import EarlyStopping


# Early Stopping to Prevent Overfitting
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights = True)

In [77]:
# 5. Train the model
batch_size = 64
epochs = 200
model.fit(train_generator, epochs = epochs, validation_data = val_batch,
                    class_weight=class_weight,  # Fix: Handling class imbalance
                    batch_size = batch_size)
, callbacks = [early_stop]

Epoch 1/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 60s 625ms/step - accuracy: 0.9651 - loss: 0.0094 - val_accuracy: 0.7208 - val_loss: 0.1802
Epoch 2/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 60s 627ms/step - accuracy: 0.9558 - loss: 0.0114 - val_accuracy: 0.8788 - val_loss: 0.0605
Epoch 3/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 59s 622ms/step - accuracy: 0.9711 - loss: 0.0085 - val_accuracy: 0.8303 - val_loss: 0.0984
Epoch 4/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 61s 637ms/step - accuracy: 0.9668 - loss: 0.0103 - val_accuracy: 0.7682 - val_loss: 0.1095
Epoch 5/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 60s 627ms/step - accuracy: 0.9787 - loss: 0.0049 - val_accuracy: 0.8577 - val_loss: 0.0867
Epoch 6/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 59s 621ms/step - accuracy: 0.9830 - loss: 0.0047 - val_accuracy: 0.8599 - val_loss: 0.0787
Epoch 7/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 61s 639ms/step - accuracy: 0.9774 - loss: 0.0058 - val_accuracy: 0.7998 - val_loss: 0.1078
Epoch 8/200
89/89 ━━━━━━━━━━━━━━━━━━━━ 61s 643ms/step - accuracy: 0.9561 - loss: 0.0113 - 

('callbacks', '=', '[early_stop]')

In [78]:
from sklearn.metrics import classification_report, accuracy_score

# Predict on validation data
y_pred_prob = model.predict(val_batch)  
y_pred = np.argmax(y_pred_prob, axis=1)

# Get true labels from the generator
y_true = val_batch.classes  

# Get class labels
labels = list(val_batch.class_indices.keys())

# Calculate validation accuracy
val_accuracy = accuracy_score(y_true, y_pred)

# Print classification report
print("Validation Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=labels))

# Print validation accuracy
print(f"\nValidation Accuracy: {val_accuracy:.4f}")

# Get training accuracy from history object
if 'accuracy' in history.history and 'val_accuracy' in history.history:
    train_accuracy = history.history['accuracy'][-1]  # Last epoch train accuracy
    print(f"Train Accuracy: {train_accuracy:.4f}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 151ms/step
Validation Classification Report:

                     precision    recall  f1-score   support

          Cardboard       0.07      0.12      0.09        92
      Food Organics       0.07      0.06      0.07        82
              Glass       0.09      0.10      0.09        84
              Metal       0.12      0.13      0.13       158
Miscellaneous Trash       0.09      0.07      0.08        99
              Paper       0.13      0.14      0.14       100
            Plastic       0.19      0.14      0.16       184
      Textile Trash       0.06      0.05      0.05        63
         Vegetation       0.10      0.11      0.11        87

           accuracy                           0.11       949
          macro avg       0.10      0.10      0.10       949
       weighted avg       0.11      0.11      0.11       949


Validation Accuracy: 0.1096


NameError: name 'history' is not defined